In [ ]:
using BenchmarkTools
using Random
using StatsBase

In [ ]:
Random.seed!(20121020);

In [ ]:
element_ = Array('A':'Z')

element_ = string.(element_);

In [ ]:
n_element = length(element_);

In [ ]:
element_score_ = randn(n_element);

In [ ]:
n_set_element = 3

set_element_ = sample(element_, n_set_element; replace=false)

In [ ]:
index_ = sortperm(element_score_);

In [ ]:
element_score_ = element_score_[index_];

In [ ]:
# element_ = element_[index_];

In [ ]:
set_element_ = Dict(set_element => nothing for set_element in set_element_)

In [ ]:
h_1 = [haskey(set_element_, element) for element in element_]